In [4]:
import QuantLib as ql

# Set the evaluation date
ql.Settings.instance().evaluationDate = ql.Date(3,1,2023)

# Define market data for the curve
data = {
    '1Y': 3.45,
    '2Y': 3.4,
    '3Y': 3.37,
    '4Y': 3.33,
    '5Y': 3.2775,
    '6Y': 3.235,
    '7Y': 3.205,
    '8Y': 3.1775,
    '9Y': 3.1525,
    '10Y': 3.1325,
    '12Y': 3.095,
    '15Y': 3.0275,
    '20Y': 2.92,
    '25Y': 2.815,
    '30Y': 2.6925
}
# Create a custom ibor index for the floating leg
ibor_index = ql.IborIndex(
    'STIBOR3M',           # Name of the index
    ql.Period('3M'),      # Maturity
    0,                    # Fixing days
    ql.SEKCurrency(),     # Currency
    ql.Sweden(),          # Calendar
    ql.ModifiedFollowing, # Convention
    False,                # EOM convention
    ql.Actual360()        # Daycount
)
# Create the bootstrapping instruments using helpers
swap_helpers = [ql.SwapRateHelper(
   ql.QuoteHandle(ql.SimpleQuote(rate/100.0)),   # Quote
   ql.Period(term),                              # Maturity
   ql.Sweden(),                                  # Calendar
   ql.Annual,                                    # Fixed payments
   ql.ModifiedFollowing,                         # Convention
   ql.Actual360(),                               # Daycount
ibor_index) for term, rate in data.items()]
curve = ql.PiecewiseLogLinearDiscount(0, ql.Sweden(), swap_helpers, ql.Actual360())

In [9]:
# Link the zero rate curve to be used as forward and discounting
yts = ql.RelinkableYieldTermStructureHandle()
yts.linkTo(curve)
engine = ql.DiscountingSwapEngine(yts)

# Define the maturity of our swap
maturity = ql.Period("10y")
# Create a custom Ibor index for the floating leg
custom_ibor_index = ql.IborIndex(
    "Ibor",
    ql.Period("1Y"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    yts,
)
fixed_rate = 0.03269
forward_start = ql.Period("0D")
# Create the swap using the helper class MakeVanillaSwap
swap = ql.MakeVanillaSwap(
    maturity,
    custom_ibor_index,
    fixed_rate,
    forward_start,
    Nominal=10e7,
    pricingEngine=engine,
    fixedLegDayCount=ql.Actual360(),
)

In [10]:
import pandas as pd

fixed_cashflows = pd.DataFrame(
    [
        {
            "Type": "FixedPeriod",
            "accrualStart": cf.accrualStartDate().ISO(),
            "accrualEnd": cf.accrualEndDate().ISO(),
            "paymentDate": cf.date().ISO(),
            "df": curve.discount(cf.accrualEndDate()),
            "rate": cf.rate(),
            "cashflow": cf.amount(),
            "npv": -curve.discount(cf.accrualEndDate()) * cf.amount(),
        }
        for cf in map(ql.as_fixed_rate_coupon, swap.leg(0))
    ]
)

float_cashflows = pd.DataFrame(
    [
        {
            "Type": "FloatPeriod",
            "accrualStart": cf.accrualStartDate().ISO(),
            "accrualEnd": cf.accrualEndDate().ISO(),
            "paymentDate": cf.date().ISO(),
            "df": curve.discount(cf.accrualEndDate()),
            "rate": cf.rate(),
            "cashflow": cf.amount(),
            "npv": curve.discount(cf.accrualEndDate()) * cf.amount(),
        }
        for cf in map(ql.as_floating_rate_coupon, swap.leg(1))
    ]
)

ql_cashflows = pd.concat([fixed_cashflows, float_cashflows])

In [109]:
import QuantLib as ql
import pandas as pd
# Begin by setting the valuation date of which the cap and the floor should be priced at
ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2022)
# Then we initialize the curve we want to use for discounting and forecasting
discount_factors = [1, 0.965, 0.94]  # discount factors
dates = [
    ql.Date(1, 1, 2022),
    ql.Date(1, 1, 2023),
    ql.Date(1, 1, 2024),
]  # maturity dates of the discount factors
day_counter = ql.Actual360()
# Note that we will not strip a curve here, but simply use the discount factors and the dates defined above
# By default QuantLib DiscountCurve will log linearly interpolate between the points.
discount_curve = ql.DiscountCurve(dates, discount_factors, day_counter)
# The curve will note be linked in case we want to update the quotes later on
discount_handle = ql.YieldTermStructureHandle(discount_curve)
start_date = ql.Date(1, 1, 2022)
end_date = start_date + ql.Period(12, ql.Months)

# We define the schedule of the cap and floor
schedule = ql.Schedule(
    start_date,                 # Start date of payments
    end_date,                   # End date of payments
    ql.Period(2, ql.Months),    # frequency of payments
    ql.Sweden(),                # Calendar for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.DateGeneration.Backward, # Date generation rule for generating the schedule
    False,                      # End of month rule
)

# Create a custom index to track the payments correctly, specifically fixing days.
custom_discount_index= ql.IborIndex(
    "MyIndex",
    ql.Period("1m"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    discount_handle,
)
custom_discount_index.addFixing(ql.Date(3,1, 2022), 0.03)
# As you have noted by now, the pricing of caps and floors involves creating a floating leg
ibor_leg_discount = ql.IborLeg([1e6], schedule, custom_discount_index, isInArrears=False)

In [110]:
leg = ql.FixedRateLeg(schedule, ql.Actual360(), [1e6], [0.05], ql.Following)
swap = ql.Swap(leg, ibor_leg_discount)
_ = ql.DiscountingSwapEngine(discount_handle)
swap.setPricingEngine(_)

In [112]:
import pandas as pd

fixed_cashflows = pd.DataFrame(
    [
        {
            "Type": "FixedPeriod",
            "accrualStart": cf.accrualStartDate().ISO(),
            "accrualEnd": cf.accrualEndDate().ISO(),
            "paymentDate": cf.date().ISO(),
            "rate": cf.rate(),
            "cashflow": cf.amount(),
        }
        for cf in map(ql.as_fixed_rate_coupon, swap.leg(0))
    ]
)


float_cashflows = pd.DataFrame(
    [
        {
            "Type": "FloatPeriod",
            "accrualStart": cf.accrualStartDate().ISO(),
            "accrualEnd": cf.accrualEndDate().ISO(),
            "paymentDate": cf.date().ISO(),
            "rate": cf.rate(),
            "cashflow": cf.amount(),
        }
        for cf in map(ql.as_floating_rate_coupon, swap.leg(1))
    ]
)

ql_cashflows = pd.concat([fixed_cashflows, float_cashflows])
ql_cashflows.to_clipboard()

In [63]:

strike = [0.025]
cap_discount = ql.Cap(ibor_leg_discount, strike)

# The final step is to define a volatility surface, we will use a constant volatility for simplicity
volatility = ql.QuoteHandle(ql.SimpleQuote(0.5))

# Input our discounting and forecasting curve together with our volatility surface to the engine
engine = ql.BlackCapFloorEngine(discount_handle, volatility)
cap_discount.setPricingEngine(engine)
print(cap_discount.NPV())

10880.76034601696


In [23]:
schedule_dates = schedule.dates()

display_result = lambda _: pd.DataFrame(
    {
        "price": _.optionletsPrice(),
        "discount_factor": _.optionletsDiscountFactor(),
        "cap_rate": _.capRates(),
        "atm_forward": _.optionletsAtmForward(),
        "std_dev": _.optionletsStdDev(),
        "accrual_start": schedule_dates[:-1],
        "accrual_end": schedule_dates[1:],
    }
)

display_result(cap_discount)

RuntimeError: pricer not set

In [24]:
# As you have noted by now, the pricing of caps and floors involves creating a floating leg
dates = [cf.date().ISO() for cf in ibor_leg_discount]
amount = [cf.amount() for cf in ibor_leg_discount]

pd.DataFrame({"Date": dates, "Amount": amount})


RuntimeError: pricer not set

In [57]:
# As you have noted by now, the pricing of caps and floors involves creating a floating leg
dates = [cf.date().ISO() for cf in ibor_leg_discount]
amount = [cf.amount() for cf in ibor_leg_discount]

pd.DataFrame({"Date": dates, "Amount": amount})


,Date,Amount
0,2022-03-01,5579.202336
1,2022-05-02,6070.088536
2,2022-07-01,5873.705307
3,2022-09-01,6070.088536
4,2022-11-01,5971.892130
5,2023-01-02,6044.237135


In [42]:
amo

'2022-04-01'

In [39]:
_ = ibor_leg_discount[0]

Date(1,4,2022)

In [117]:
issue_date = ql.Date(31, 1, 2011)
maturity_date = ql.Date(31, 1, 2016)
coupon_rate = 6.23/100 
face_value = 250000
calendar = ql.NullCalendar() 
calendar.addHoliday(ql.Date(31,7,2011))
calendar.addHoliday(ql.Date(30,7,2011))
day_count = ql.Actual365Fixed()
payment_frequency = ql.Semiannual
schedule = ql.Schedule(issue_date,maturity_date,ql.Period(payment_frequency),calendar,ql.ModifiedFollowing,ql.ModifiedFollowing,ql.DateGeneration.Forward,False)
forcast_curve = ql.RelinkableYieldTermStructureHandle()
curve = ql.FlatForward(0,ql.NullCalendar(),coupon_rate,ql.Actual365Fixed(),ql.Continuous)
forcast_curve.linkTo(curve)
index = ql.IborIndex("myindex",ql.Period(ql.Semiannual),0, ql.INRCurrency(),ql.NullCalendar(),ql.ModifiedFollowing,False,day_count,forcast_curve) 
index.clearFixings()                  
adates = [a for a in schedule]
arates = [0.055,0.0623,0.0623,0.0623,0.0623,0.0623,0.0623,0.0623,0.0623,0.0623,0.0623]
index.addFixings(adates, arates, True) 
iborleg = ql.IborLeg([face_value],schedule,index,day_count,ql.ModifiedFollowing,isInArrears=True)

#Coupon pricer
pricer = ql.BlackIborCouponPricer()

volatility = 1.0

vol = ql.ConstantOptionletVolatility(0,
                                     calendar,
                                      ql.ModifiedFollowing,
                                      volatility,
                                      ql.Actual365Fixed())

pricer.setCapletVolatility(ql.OptionletVolatilityStructureHandle(vol))

ql.setCouponPricer(iborleg, pricer)
cashflows = pd.DataFrame([(a.accrualStartDate().ISO(),a.accrualEndDate().ISO(),a.date().ISO(),a.amount(),a.accrualDays(),a.rate())
                    for a in [ql.as_coupon(c) for c in iborleg]])
cashflows.columns =["Start_Date","End_Date","Payment_Date","Amount","No_of_Days","Rate"]
cashflows.to_clipboard()

In [118]:
cashflows

,Start_Date,End_Date,Payment_Date,Amount,No_of_Days,Rate
0,2011-01-31,2011-07-29,2011-07-29,7638.150685,179,0.0623
1,2011-07-29,2012-01-31,2012-01-31,7936.849315,186,0.0623
2,2012-01-31,2012-07-31,2012-07-31,7766.164384,182,0.0623
3,2012-07-31,2013-01-31,2013-01-31,7851.506849,184,0.0623
4,2013-01-31,2013-07-31,2013-07-31,7723.493151,181,0.0623
5,2013-07-31,2014-01-31,2014-01-31,7851.506849,184,0.0623
6,2014-01-31,2014-07-31,2014-07-31,7723.493151,181,0.0623
7,2014-07-31,2015-01-31,2015-01-31,7851.506849,184,0.0623
8,2015-01-31,2015-07-31,2015-07-31,7723.493151,181,0.0623
9,2015-07-31,2016-01-31,2016-01-31,7851.506849,184,0.0623


In [21]:
adates

[Date(31,1,2011),
 Date(29,7,2011),
 Date(31,1,2012),
 Date(31,7,2012),
 Date(31,1,2013),
 Date(31,7,2013),
 Date(31,1,2014),
 Date(31,7,2014),
 Date(31,1,2015),
 Date(31,7,2015),
 Date(31,1,2016)]

In [79]:
# Set the evaluation date
ql.Settings.instance().evaluationDate = ql.Date(1,1,2023)
crv = ql.FlatForward(0, ql.TARGET(), 0.01, ql.Actual360())
yts = ql.YieldTermStructureHandle(crv)
index = ql.Euribor3M(yts)

schedule = ql.MakeSchedule(ql.Date(1,1,2023), ql.Date(15,6,2024), ql.Period('6M'))

leg = ql.IborLeg([100], schedule, index, ql.Actual360(), ql.ModifiedFollowing, isInArrears=True)

volatility = 0.10
vol = ql.ConstantOptionletVolatility(2, ql.TARGET(), ql.Following, volatility, ql.Actual360())
pricer = ql.BlackIborCouponPricer(ql.OptionletVolatilityStructureHandle(vol))
ql.setCouponPricer(leg, pricer)

npv = ql.CashFlows.npv(leg, yts, True)
print(f"LEG NPV: {npv:,.2f}")

RuntimeError: negative time (-0.00277778) given